In [1]:
from string import ascii_lowercase
import requests
from bs4 import BeautifulSoup
import pandas as pd
import collections
import re
import csv

### Get list of fighters & associated links to details

In [ ]:
page_urls =[]
fighters = {}

fighter_base_url = 'http://www.fightmetric.com/statistics/fighters?char='
for alphabet in ascii_lowercase:
    page_urls.append('{}{}&page=all'.format(fighter_base_url,alphabet))

page_urls_short = page_urls[0:2]

for page_url in page_urls:
    i = 2
    r = requests.get(page_url)
    soup = BeautifulSoup(r.text, 'html.parser')
    fighter_ref_top = soup.find_all('tr', class_='b-statistics__table-row')
    
    while i < (len(fighter_ref_top)):
        fighter_ref = fighter_ref_top[i].find_all('td', class_='b-statistics__table-col')
        fighter_first = fighter_ref[0].text.strip("\n")
        fighter_last = fighter_ref[1].text.strip("\n")
        fighter_nick = fighter_ref[2].text.strip("\n")
        fighter_name = '{} {}_{}'.format(fighter_first, fighter_last, fighter_nick)
        fighter_url = fighter_ref[0].find('a')['href']
        fighters[fighter_name] = fighter_url
        i+=1

### Write fighter reference pages (URLs) to CSV

In [ ]:
csv_file = 'data/fighter_list.csv'
with open (csv_file, 'w', newline='') as csvfile:
    fighterwriter = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    for key in fighters:
        fighterwriter.writerow([key, fighters[key]])

### Get fighter stats
**
SLpM - Significant Strikes Landed per Minute<br>
Str. Acc. - Significant Striking Accuracy<br>
SApM - Significant Strikes Absorbed per Minute<br>
Str. Def. - Significant Strike Defence (the % of opponents strikes that did not land)<br>
TD Avg. - Average Takedowns Landed per 15 minutes<br>
TD Acc. - Takedown Accuracy<br>
TD Def. - Takedown Defense (the % of opponents TD attempts that did not land)<br>
Sub. Avg. - Average Submissions Attempted per 15 minutes<br>
**

In [ ]:
csv_file = 'data/fighter_list.csv'
csv_file_out = 'data/fighter_details.csv'
df = pd.read_csv(csv_file)
# Breaking into smaller chunks
fighters_1 = df['Fighter Name'][:]
urls_1 = df['Fighter URL'][:]
count = 0

for url in urls_1:
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    fighter_record = soup.find('span', class_='b-content__title-record')
    fighter_stats = soup.find_all('li', class_='b-list__box-list-item b-list__box-list-item_type_block')
    fighter_attrbs = collections.OrderedDict([('HEIGHT', ""), ('WEIGHT', ""), ('REACH', ""), ('STANCE', ""), ('DOB', ""),\
                                              ('SLpM', ""), ('Str_Acc', ""), ('SApM', ""), ('Str_Def', ""), ('DUMMY', ""),\
                                              ('TD_Avg', ""), ('TD_Acc', ""), ('TD_Def', ""), ('Sub_Avg', "")])
    
    i=0
    for key in fighter_attrbs:
        fighter_attrbs[key] = re.split('\n\s*', fighter_stats[i].contents[2])[1]
        i+=1
    fighter_attrbs['RECORD'] = re.split(':\s*',fighter_record.text)[1].rstrip()
    
    with open (csv_file_out, 'a', newline='') as csvfile:
        fighterstats = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_MINIMAL)
        fighterstats.writerow([fighters_1[count], fighter_attrbs['RECORD'], fighter_attrbs['HEIGHT'], fighter_attrbs['WEIGHT'],\
                              fighter_attrbs['REACH'], fighter_attrbs['STANCE'], fighter_attrbs['DOB'],\
                             fighter_attrbs['SLpM'], fighter_attrbs['Str_Acc'], fighter_attrbs['SApM'], \
                             fighter_attrbs['Str_Def'], fighter_attrbs['TD_Avg'], fighter_attrbs['TD_Acc'], \
                             fighter_attrbs['TD_Def'], fighter_attrbs['Sub_Avg'], urls_1[count]])
    count+=1

In [3]:
df = pd.read_csv('data/fighter_details.csv')
# df.head()

### --------- Consolidated script ---------
***Needs to be completed***

In [ ]:
from string import ascii_lowercase
import requests
from bs4 import BeautifulSoup
import pandas as pd
import collections
import re

def get_fighter_stats(fighter_url):
    r = requests.get(fighter_url)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    fighter_record = soup.find('span', class_='b-content__title-record')
    fighter_stats = soup.find_all('li', class_='b-list__box-list-item b-list__box-list-item_type_block')
    
    '''
    SLpM - Significant Strikes Landed per Minute
    Str. Acc. - Significant Striking Accuracy
    SApM - Significant Strikes Absorbed per Minute
    Str. Def. - Significant Strike Defence (the % of opponents strikes that did not land)
    TD Avg. - Average Takedowns Landed per 15 minutes
    TD Acc. - Takedown Accuracy
    TD Def. - Takedown Defense (the % of opponents TD attempts that did not land)
    Sub. Avg. - Average Submissions Attempted per 15 minutes
    '''
    fighter_attrbs = collections.OrderedDict([('HEIGHT', ""), ('WEIGHT', ""), ('REACH', ""), ('STANCE', ""), ('DOB', ""),\
                                             ('SLpM', ""), ('Str_Acc', ""), ('SApM', ""), ('Str_Def', ""), ('DUMMY', ""),\
                                              ('TD_Avg', ""), ('TD_Acc', ""), ('TD_Def', ""), ('Sub_Avg', "")])
    i=0
    for key in fighter_attrbs:
        fighter_attrbs[key] = re.split('\n\s*', fighter_stats[i].contents[2])[1]
        i+=1
    fighter_attrbs['RECORD'] = re.split(':\s*',fighter_record.text)[1].rstrip()
    return(fighter_attrbs)

def get_fighter_list():
    ############# Need to populate #############
    pass

def main():
    fighter_list = get_fighter_list()
#     fighter_attrbs = get_fighter_stats ('http://www.fightmetric.com/fighter-details/634e2fb70bde3fd5')
#     print (fighter_attrbs['TD_Def'])
        
if __name__ == "__main__":
    main()